# Photometric Zeropoints

In [1]:
from astropy.io.votable import parse_single_table
from astropy.io.votable import from_table,writeto
from astropy import units as u

import matplotlib.pyplot as plt #to plot
import math
import numpy as np
import os
from IPython.display import display,Image #to display tables in a well-looked format

## Preliminary data

Mean Extinction Coefficient and Field 01 Airmass:

\begin{align*}
    K_Y &= 0.1228\\
    X_Y &= 1.405\\
    K_u &= 0.3913\\
    X_u &= 1.061
\end{align*}

# ZeroPoint Calculator Function

In [5]:
def zeropoint(decam_filter,ccd,K,X,exptime): #decam_filter=u or Y , ccd=S or N , K=extinction coeff , X=airmass
    os.chdir('/home/alex-cfrd/Escritorio/SEXtractor/filter_'+decam_filter+'/') #move to the directory
    #number of CCDs
    if ccd=='S':
        number_ccds = 31
    elif ccd=='N':
        number_ccds = 30
    #LOAD CATALOG
    if decam_filter=='u':
        stand = parse_single_table('gsc_'+ccd+decam_filter+'.vot').to_table() #convert the vot file to table
    else:
        stand = parse_single_table('des_'+ccd+decam_filter+'.vot').to_table()
    #REMOVE OUTLAYERS
    stand.remove_rows(stand['FLUX_AUTO']<=0.0) #remove outlayers
    stand.remove_rows(stand['FLUX_APER']<=0.0) #remove outlayers
    #ZEROPOINT CATALOG COLUMN
    stand.add_column(1.0*u.mag,name='ZEROPOINT') #add a preliminary zeropoint column
    stand.add_column(ccd,name='CCD',index=1) #add a ccd column (identify S and N)
    #FIX STRING TYPE COLUMNS
    for col in stand.itercols():
        if col.dtype.kind in 'SU':
            stand.replace_column(col.name, col.astype('object'))
    #ENUMERATE ROWS AND FIX EXTENSION CCDs
    for i in range(len(stand['NUMBER'])): #for each row
        stand['NUMBER'][i] = i+1 #reorganize numerically the object number
        if ccd=='N' and stand['EXT_NUMBER'][i]==30:
            stand['CCD'][i] += str(stand['EXT_NUMBER'][i]+1)
        else:
            stand['CCD'][i] += str(stand['EXT_NUMBER'][i])
        stand['MAG_APER'][i] += 2.5*math.log10(exptime)
        stand['MAG_AUTO'][i] += 2.5*math.log10(exptime)
    #ZEROPOINTS AND ERRORS CCD SETS EMPTY LISTS
    ZP = [] #to allocate the zeropoint samples per ccd
    ZP_delta = [] #to allocate the zeropoint sample deltas per ccd
    #CALCULATE ZEROPOINTS
    for i in range(1,number_ccds+1): #for each CCD
        if decam_filter == 'u':
            mag_cat = stand[stand['EXT_NUMBER']==i]['umag'] #extract the cat_mag for CCD [Si]
            mag_ins = stand[stand['EXT_NUMBER']==i]['MAG_AUTO']-K*X #extract the sex_mag for CCD [Si] and apply atmospheric correction
            delta_cat = stand[stand['EXT_NUMBER']==i]['e_umag'] #extract the cat_mag_err for CCD [Si] 
            delta_ins = stand[stand['EXT_NUMBER']==i]['MAGERR_AUTO'] #extract sex_mag_err for CCD [Si]
        elif decam_filter == 'Y':
            mag_cat = stand[stand['EXT_NUMBER']==i]['mag_auto_y'] #extract the cat_mag for CCD [Si]
            mag_ins = stand[stand['EXT_NUMBER']==i]['MAG_AUTO']-K*X #extract the sex_mag for CCD [Si] and apply atmospheric correction
            delta_cat = stand[stand['EXT_NUMBER']==i]['magerr_auto_y'] #extract the cat_mag_err for CCD [Si] 
            delta_ins = stand[stand['EXT_NUMBER']==i]['MAGERR_AUTO'] #extract sex_mag_err for CCD [Si]
        else:
            mag_cat = stand[stand['EXT_NUMBER']==i]['mag_auto_'+decam_filter] #extract the cat_mag for CCD [Si]
            mag_ins = stand[stand['EXT_NUMBER']==i]['MAG_AUTO']-K*X #extract the sex_mag for CCD [Si] and apply atmospheric correction
            delta_cat = stand[stand['EXT_NUMBER']==i]['magerr_auto_'+decam_filter] #extract the cat_mag_err for CCD [Si] 
            delta_ins = stand[stand['EXT_NUMBER']==i]['MAGERR_AUTO'] #extract sex_mag_err for CCD [Si]
        delta = np.sqrt(delta_cat**2 + delta_ins**2) #error propagation
        zp = mag_cat - mag_ins #calculate zeropoint
        for j in range(len(zp)):
            if zp[j]<20:
                zp[j]=np.mean(zp)
        ZP.append(zp) #append CCD [Si] sample
        ZP_delta.append(delta) #append deltas
    #APPEND CALCULATED ZEROPOINTS TO THE CATALOG
    for i in range(len(stand['ZEROPOINT'])): #for each row
        ext = stand['EXT_NUMBER'][i]-1 #extension number
        stand['ZEROPOINT'][i] = round(np.median(ZP[ext]),8) #append the median zeropoint
    #ZEROPOINT ERROR COLUMNS
    stand.add_column(1.0*u.mag,name='ZP_STD')
    stand.add_column(1.0*u.mag,name='ZP_MAG_ERR')
    stand.add_column(1.0*u.mag,name='ZP_TOTAL_ERR')
    #ZEROPOINTS ERROR EMPTY LISTS PER CCD
    ZP_MAG_ERR = [] #mag error (catalog error + sextractor error)
    ZP_STD = [] #standard deviation of the ccd set (sigma of the zeropoints in a ccd)
    ZP_TOTAL_ERR = [] #cuadratur sum of ZP_MAG_ERR and ZP_STD
    #CALCULATE ZP_STD AND ZP_TOTAL_ERR AND APPEND TO THE EMPTY LISTS
    for i in range(len(ZP)):
        std = np.std(ZP[i]) #calculate std deviation of zeropoint sample per ccd
        if (len(ZP[i])%2)!=0: #if list have impar elements
            index = list(ZP[i]).index(np.median(ZP[i])) #calculate index of the delta of the median z´
            mags_delta = ZP_delta[i][index] #extract delta of the median zp
        if (len(ZP[i])%2)==0: #if list have par elements
            index_1 = int((len(ZP[i])/2)-1) #extract central elements
            index_2 = int(len(ZP[i])/2) #extract central elements
            mags_delta = max(ZP_delta[i][index_1],ZP_delta[i][index_2]) #extract the max element
        total_error = math.sqrt(std**2 + mags_delta**2) #quadrature sum
        ZP_MAG_ERR.append(mags_delta) #append to list
        ZP_STD.append(std) #append to list
        ZP_TOTAL_ERR.append(total_error) #append to list
    #APPEND CALCULATED ZEROPOINT ERRORS TO THE CATALOG
    for i in range(len(stand['ZEROPOINT'])): #for each row
        ext = stand['EXT_NUMBER'][i]-1 #extension number
        stand['ZP_STD'][i] = round(np.median(ZP_STD[ext]),8) #append the ZP std
        stand['ZP_MAG_ERR'][i] = round(np.median(ZP_MAG_ERR[ext]),8) #append the zp mag error
        stand['ZP_TOTAL_ERR'][i] = round(np.median(ZP_TOTAL_ERR[ext]),8) #append the quad total error
    #STATISTICS PER CCD
    print('#                          ZEROPOINT STATISTICS '+ccd+decam_filter+'                        ')
    print('#-----------------------------------------------------------------------------')
    print('#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS')
    print('#-----------------------------------------------------------------------------')
    for i in range(1,number_ccds+1):
        mean = str(np.mean(ZP[i-1])) #mean zp
        median = '  '+str(np.median(ZP[i-1])) #median zp
        minimum = '    '+str(np.min(ZP[i-1])) #min zp
        maximum = '   '+str(np.max(ZP[i-1])) #max zp
        sigma = '  '+str(np.std(ZP[i-1])) #std zp
        mag_err = '  '+str(ZP_MAG_ERR[i-1]) #mag_err zp
        quad_err = '  '+str(ZP_TOTAL_ERR[i-1]) #quad sum
        stars = '   '+str(len(ZP[i-1]))
        if i<10:
            print('['+ccd+str(i)+']   '+mean[:7]+median[:9]+minimum[:9]+maximum[:9]+sigma[:9]+mag_err[:9]+quad_err[:9]+stars)
        elif ccd=='N' and i==30:
            print('['+ccd+str(i+1)+']  '+mean[:7]+median[:9]+minimum[:9]+maximum[:9]+sigma[:9]+mag_err[:9]+quad_err[:9]+stars)
        else:
            print('['+ccd+str(i)+']  '+mean[:7]+median[:9]+minimum[:9]+maximum[:9]+sigma[:9]+mag_err[:9]+quad_err[:9]+stars)
    #REMOVE COLUMN
    stand.remove_column('EXT_NUMBER')
    return stand

## Filter $Y$ [S1-S31] CCDs

In [11]:
ZP_SY = zeropoint('Y','S',0.1228,1.405,1) #photometry were made in counts per second image

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS SY                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[S1]   24.0732  24.0739    23.95   24.408  0.06683  0.00068  0.06683   35
[S2]   24.1184  24.0574    23.88   25.707  0.27844  0.00286  0.27846   52
[S3]   24.0536  24.0332    23.91   24.785  0.13396  0.00327  0.13400   46
[S4]   24.0894  24.0735    23.92   24.536  0.11577  0.00276  0.11580   44
[S5]   24.0754  24.0671    23.92   24.490  0.08300  0.00413  0.08310   44
[S6]   24.0689  24.0759    23.86   24.477  0.08349  0.00383  0.08358   44
[S7]   24.0993  24.0872    23.81   24.474  0.10515  0.00065  0.10515   47
[S8]   24.0923  24.0912    23.84   24.490  0.08860  0.00283  0.08865   47
[S9]   24.0600  24.0489    23.92   24.219  0.06019  0.00272  0.06026   41
[S10]  24.1061  24.0855

## Filter $Y$ [N1-N31] CCDs (no [N31])

In [12]:
ZP_NY =zeropoint('Y','N',0.1228,1.405,1) #photometry were made in counts per second image

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS NY                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[N1]   24.1003  24.0957    24.01   24.336  0.05590  0.00296  0.05598   42
[N2]   24.1326  24.0927    23.98   24.875  0.15459  0.00520  0.15467   48
[N3]   24.1038  24.0628    23.95   24.541  0.13519  0.00154  0.13520   48
[N4]   24.1807  24.0658    23.91   26.049  0.35600  0.00243  0.35601   47
[N5]   24.1008  24.0703    23.97   24.489  0.10988  0.00279  0.10991   44
[N6]   24.1292  24.0844    23.80   24.718  0.19066  0.00131  0.19067   39
[N7]   24.1517  24.1302    24.03   24.423  0.08654  0.00342  0.08661   50
[N8]   24.1392  24.1116    23.99   24.949  0.14825  0.00170  0.14826   39
[N9]   24.0960  24.0704    23.91   25.197  0.18223  0.00292  0.18226   48
[N10]  24.0880  24.0499

## General Values Filter $Y$

\begin{align*}
    ZP_{\text{ median}}&=24.07685\pm 0.10457\\
    ZP_{\text{ mean}}&=24.0788401639\\
    ZP_{\text{ min}}&=24.0188\\
    ZP_{\text{ max}}&=24.13029\\
    ZP_{\text{ std}}&=0.021615574\\
\end{align*}

Due to the error propagation, and addopting the median as the definitive value:

\begin{align*}
    ZP &= 24.07685\pm\sqrt{0.10457^2+ZP_{\text{ std}}^2}\\
    &= 24.07685\pm\sqrt{0.10457^2+0.021615574^2}\\
    ZP &= 24.07685\pm 0.1067807
\end{align*}

## Filter $u$ [S1-S31] CCDs

In [13]:
ZP_Su =zeropoint('u','S',0.3913,1.061,1) #photometry were made in counts per second image

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Su                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[S1]   23.5060  23.5043    23.33   23.722  0.08541  0.03811  0.09353   23
[S2]   23.5530  23.5128    23.43   24.362  0.20126  0.00805  0.20142   19
[S3]   23.4756  23.4836    23.23   23.638  0.08473  0.01603  0.08624   28
[S4]   23.5143  23.5020    23.31   23.907  0.12583  0.01113  0.12632   23
[S5]   23.5206  23.5276    23.13   23.669  0.12771  0.05803  0.14027   13
[S6]   23.4494  23.4932    22.87   23.837  0.17347  0.03500  0.17697   31
[S7]   23.4847  23.4926    23.00   23.667  0.13395  0.03403  0.13821   26
[S8]   23.4401  23.4592    23.21   23.552  0.08714  0.04001  0.09589   22
[S9]   23.5296  23.5245    22.92   23.958  0.19515  0.07302  0.20837   20
[S10]  23.4533  23.4803

## Filter $u$ [N1-N31] CCDs (no [N31])

In [14]:
ZP_Nu =zeropoint('u','N',0.3913,1.061,1) #photometry were made in counts per second image

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Nu                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[N1]   23.4005  23.4210    22.98   23.739  0.15385  0.02601  0.15604   24
[N2]   23.4050  23.4357    23.02   23.586  0.12644  0.03204  0.13043   27
[N3]   23.3838  23.4296    22.80   23.597  0.16673  0.09602  0.19240   28
[N4]   23.4534  23.4688    23.23   23.555  0.08599  0.01607  0.08748   17
[N5]   23.4013  23.4564    22.75   23.644  0.17558  0.03401  0.17884   28
[N6]   23.3841  23.4349    22.98   23.582  0.14615  0.07501  0.16428   28
[N7]   23.3742  23.4408    22.63   23.541  0.21857  0.01808  0.21931   28
[N8]   23.4476  23.4684    23.06   23.751  0.12093  0.04403  0.12870   28
[N9]   23.4851  23.4922    23.15   23.691  0.12918  0.09702  0.16156   24
[N10]  23.4387  23.4679

## General Values Filter $u$

\begin{align*}
    ZP_{\text{ median}}&=23.46297\pm 0.13655\\
    ZP_{\text{ mean}}&=23.46886836\\
    ZP_{\text{ min}}&=23.35337\\
    ZP_{\text{ max}}&=23.56122\\
    ZP_{\text{ std}}&=0.0484469779\\
\end{align*}

Due to the error propagation, and addopting the median as the definitive value:

\begin{align*}
    ZP &= 23.46297\pm\sqrt{0.13655^2+ZP_{\text{ std}}^2}\\
    &= 23.46297\pm\sqrt{0.13655^2+0.0484469779^2}\\
    ZP &= 23.46297\pm 0.144889655
\end{align*}

## Filter $g$ [S1-S31] CCDs

In [10]:
ZP_Sg = zeropoint('g','S',0.20,1.01,300)

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Sg                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[S1]   25.6088  25.4573    25.39   26.482  0.33125  0.00057  0.33125   23
[S2]   25.5298  25.4208    25.30   26.538  0.26729  0.00080  0.26729   25
[S3]   25.5777  25.3737    25.21   26.813  0.43352  0.00035  0.43352   32
[S4]   25.4619  25.3837    25.06   26.520  0.27368  0.00085  0.27368   26
[S5]   25.4129  25.3556    25.28   26.427  0.23934  0.00095  0.23934   20
[S6]   25.4733  25.4033    25.06   26.441  0.28642  0.00104  0.28643   25
[S7]   25.7645  25.7826    25.34   26.699  0.24212  0.00107  0.24213   31
[S8]   25.5331  25.4333    25.33   26.250  0.24496  0.00092  0.24496   31
[S9]   25.4091  25.3995    24.99   25.804  0.13827  0.00070  0.13827   20
[S10]  25.4332  25.3666

## Filter $g$ [N1-N31] CCDs (no [N31])

In [56]:
ZP_Ng = zeropoint('g','N',0.20,1.01,300)

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Ng                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[N1]   25.5053  25.4408    25.37   26.284  0.22154  0.00070  0.22154   26
[N2]   25.4543  25.4143    24.32   26.194  0.27348  0.00092  0.27348   34
[N3]   25.4638  25.3780    25.30   26.500  0.23698  0.00108  0.23698   29
[N4]   25.4073  25.3900    25.28   25.686  0.08558  0.00039  0.08558   23
[N5]   25.4762  25.3926    25.32   26.433  0.22478  0.00070  0.22478   27
[N6]   25.5718  25.4042    25.30   26.760  0.38843  0.00108  0.38843   27
[N7]   25.4399  25.4380    24.44   26.175  0.25477  0.00073  0.25477   25
[N8]   25.5139  25.4272    25.38   26.858  0.29555  0.00108  0.29555   24
[N9]   25.5481  25.4059    25.34   26.402  0.32593  0.00109  0.32593   25
[N10]  25.5017  25.3829

## Filter $r$ [S1-S31] CCDs

In [57]:
ZP_Sr = zeropoint('r','S',0.10,1.065,300)

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Sr                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[S1]   25.4073  25.4058    24.93   26.264  0.22939  0.00113  0.22939   36
[S2]   25.4151  25.3735    25.05   25.972  0.20254  0.00169  0.20255   40
[S3]   25.4454  25.4037    25.02   26.436  0.27623  0.00048  0.27623   43
[S4]   25.3966  25.3765    25.07   25.903  0.16089  0.00058  0.16089   45
[S5]   25.4601  25.4125    24.92   26.146  0.22520  0.00057  0.22520   44
[S6]   25.4525  25.4056    25.05   25.961  0.16186  0.00121  0.16186   36
[S7]   25.4712  25.4400    25.00   26.488  0.24603  0.00075  0.24603   50
[S8]   25.4752  25.3731    25.06   26.975  0.37041  0.00114  0.37041   46
[S9]   25.4126  25.3496    25.10   26.258  0.22339  0.00076  0.22339   33
[S10]  25.4135  25.3731

## Filter $r$ [N1-N31] CCDs (no [N31])

In [58]:
ZP_Nr = zeropoint('r','N',0.10,1.065,300)

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Nr                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[N1]   25.4403  25.3708    25.15   26.155  0.21893  0.00056  0.21893   39
[N2]   25.3890  25.3581    25.00   26.315  0.21908  0.00098  0.21908   33
[N3]   25.3703  25.2989    25.11   26.485  0.22397  0.00051  0.22397   53
[N4]   25.5292  25.3985    25.13   26.391  0.32707  0.00064  0.32707   37
[N5]   25.4806  25.3826    24.94   26.681  0.33785  0.00059  0.33785   41
[N6]   25.4970  25.4443    25.03   26.359  0.27871  0.00073  0.27871   31
[N7]   25.4812  25.4800    25.15   26.275  0.22026  0.00050  0.22027   39
[N8]   25.4948  25.4431    24.99   26.444  0.34028  0.00086  0.34029   29
[N9]   25.3931  25.3559    25.10   26.041  0.18330  0.00084  0.18331   39
[N10]  25.3942  25.3783

## Filter $i$ [S1-S31] CCDs

In [59]:
ZP_Si = zeropoint('i','S',0.06,1.01,360)

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Si                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[S1]   25.5103  25.4051    25.35   26.272  0.24551  0.00099  0.24552   43
[S2]   25.4142  25.3804    25.18   26.048  0.13911  0.00111  0.13911   56
[S3]   25.4228  25.3694    24.98   26.046  0.17291  0.00099  0.17292   52
[S4]   25.3848  25.3625    25.30   26.105  0.12050  0.00158  0.12051   56
[S5]   25.4367  25.3749    25.32   26.039  0.16309  0.00171  0.16310   57
[S6]   25.3896  25.3905    25.14   26.111  0.14329  0.00104  0.14330   45
[S7]   25.4571  25.4262    25.03   26.229  0.18490  0.00051  0.18490   51
[S8]   25.4411  25.4011    25.31   26.019  0.13268  0.00108  0.13268   52
[S9]   25.4137  25.3834    25.12   25.977  0.13816  0.00141  0.13817   43
[S10]  25.4185  25.3749

## Filter $i$ [N1-N31] CCDs (no [N31])

In [60]:
ZP_Ni = zeropoint('i','N',0.06,1.01,360)

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Ni                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[N1]   25.4803  25.4152    25.35   26.105  0.18423  0.00119  0.18423   44
[N2]   25.4511  25.3943    25.34   26.263  0.19370  0.00171  0.19371   51
[N3]   25.4393  25.3799    25.16   26.210  0.17503  0.00115  0.17504   65
[N4]   25.4173  25.3811    23.46   26.418  0.38211  0.00072  0.38211   43
[N5]   25.4282  25.3906    25.26   26.090  0.14282  0.00089  0.14282   53
[N6]   25.4102  25.3953    25.24   25.650  0.07487  0.00119  0.07488   39
[N7]   25.4899  25.4242    25.37   26.880  0.24400  0.00060  0.24400   49
[N8]   25.5261  25.4148    25.21   26.442  0.28045  0.00178  0.28045   40
[N9]   25.3924  25.3924    25.39   25.392  0.0  0.00170  0.00170   1
[N10]  25.4738  25.3811    2

## Filter $z$ [S1-S31] CCDs

In [61]:
ZP_Sz = zeropoint('z','S',0.07,1.046,360)

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Sz                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[S1]   25.1902  25.1475    25.04   25.658  0.13494  0.00148  0.13495   36
[S2]   25.1592  25.1315    25.06   25.464  0.08614  0.00182  0.08615   52
[S3]   25.0895  25.0751    24.93   25.695  0.10507  0.00131  0.10508   44
[S4]   25.1152  25.1101    25.01   25.442  0.08075  0.00154  0.08077   53
[S5]   25.1504  25.1172    25.02   25.709  0.12112  0.00194  0.12114   55
[S6]   25.1567  25.1259    25.04   25.564  0.11650  0.00084  0.11650   44
[S7]   25.1242  25.1480    23.47   25.529  0.24909  0.00095  0.24909   49
[S8]   25.1767  25.1463    25.05   25.883  0.13254  0.00053  0.13254   49
[S9]   25.1228  25.1146    25.04   25.352  0.06053  0.00129  0.06054   40
[S10]  25.1458  25.1103

## Filter $z$ [N1-N31] CCDs (no [N31])

In [62]:
ZP_Nz = zeropoint('z','N',0.07,1.046,360)

/home/alex-cfrd/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


#                          ZEROPOINT STATISTICS Nz                        
#-----------------------------------------------------------------------------
#CCD   ZP Mean  ZP Median  ZP Min  ZP Max  ZP Std   MAG Err  QUAD Sum  #STARS
#-----------------------------------------------------------------------------
[N1]   25.1603  25.1551    25.08   25.317  0.04199  0.00184  0.04203   40
[N2]   25.1267  25.1251    25.02   25.281  0.04768  0.00163  0.04771   49
[N3]   25.1125  25.0975    25.01   25.692  0.10198  0.00182  0.10199   52
[N4]   25.1097  25.1005    25.01   25.381  0.08361  0.00187  0.08363   36
[N5]   25.1561  25.0998    25.02   25.643  0.15467  0.00178  0.15468   42
[N6]   25.1716  25.1509    25.03   25.576  0.09167  0.00169  0.09169   39
[N7]   25.1777  25.1553    25.07   25.731  0.12365  0.00156  0.12366   42
[N8]   25.1891  25.1504    24.93   25.685  0.14548  0.00188  0.14549   30
[N9]   25.1470  25.1209    25.04   25.616  0.10870  0.00099  0.10871   44
[N10]  25.1110  25.1035